In [ ]:
import torch
import torch.nn as nn

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from torch.nn import functional as F

Источник датасета
https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

Определяем сентимент высказывания твитов: детектируется т.н. hate-speech (токсичные высказывания), используем сверточную нейронную сеть

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/texts_for_net/

/content/drive/My Drive/texts_for_net


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
test.head(2)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...


Будем использовать датасет train, разделив его на test и train, так как в тестовом нет лейблов

In [ ]:
train['label'].value_counts() #дисбаланс классов

0    29720
1     2242
Name: label, dtype: int64

In [ ]:
train.duplicated().sum()

0

In [ ]:
df_0 = train.loc[train['label'] == 0]
df_0 = df_0.sample(3500)
df_1 = train.loc[train['label'] == 1]

In [ ]:
frames = [df_0, df_1]
df = pd.concat(frames)
df = df.sample(frac = 1, random_state = 1)

In [ ]:
df.shape

(5742, 3)

In [ ]:
df_train = df.iloc[:5000]  #возьмем максимум на train-выборку
df_test = df.iloc[5000:]

Проведем предобработку текста, загрузим необходимые библиотеки

In [ ]:
import string

In [ ]:
from string import punctuation

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Удалим из английских стоп-слов (библиотеки nltk) отрицания, так как отсутсвие отрицания меняет смысл коренным образом. Удаление стоп-слов из текста, особенно удаление отрицаний, зависит от того, какая сеть будет обучаться, возможны разные решения

In [ ]:
sw = stopwords.words('english')
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
sw.remove('not')
sw.remove("don't")
sw.remove('nor')
sw.remove('no')
sw.remove('don')

In [ ]:
sw.index('ain')

138

In [ ]:
sw.index("wouldn't")

173

In [ ]:
del sw[138:174]

In [ ]:
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', '

In [ ]:
pip install pymorphy2

In [ ]:
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Осуществляем предобработку текста

In [ ]:
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [ ]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("[^a-zA-Z]"," ",txt) #убираем не-буквы
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]  #лемматизация
    return " ".join(txt)

In [ ]:
df_train['tweet'] = df_train['tweet'].apply(preprocess_text)
df_test['tweet'] = df_test['tweet'].apply(preprocess_text)

<ipython-input-276-bfbb7c8041ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['tweet'] = df_train['tweet'].apply(preprocess_text)
<ipython-input-276-bfbb7c8041ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['tweet'] = df_test['tweet'].apply(preprocess_text)


In [ ]:
df_train = df_train.drop('id', axis = 1)
df_test = df_test.drop('id', axis = 1)

In [ ]:
df_train.head()

,label,tweet
1101,0,tomorrow bihday pay
20828,1,nc monster law made republican lawmakers disen...
12705,1,intrumpsamerica extending hand amp uniting mer...
1623,0,got design ready go print arency shis ready pr...
3806,1,wants obama catch mad cow disease caught wherf...


Создаем словарь используемых слов (из обучающего датасета) для обучения сети и токенизируем их

In [ ]:
train_corpus = " ".join(df_train['tweet'])

In [ ]:
max_words = 10000 #сколько слов в словаре
max_len = 20 #сколько слов в высказывании

In [ ]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()] #убираем оставшиеся после предобработки спец-символы

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)] #вычитаем 1 для padding-a

In [ ]:
tokens_filtered_top[0:15] #самые популярные слова

['user',
 'amp',
 'love',
 'day',
 'not',
 'like',
 'happy',
 'trump',
 'people',
 'im',
 'no',
 'new',
 'u',
 'get',
 'dont']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()} #создаем словарь токенов, начиная с "1", т.к. "0" используется для padding-а

Преобразование твитов в последовательности токенов, которые может прочесть сеть

In [ ]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return result[-maxlen:] + padding

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["tweet"]], dtype=np.int32)

Преобразование данных в датасет для загрузки в сеть:


In [ ]:
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [ ]:
batch_size=512

Загружаем данные с помощью дата-лоадера

In [ ]:
train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Создаем сеть

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim = 128, out_channel = 128, num_classes = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
        self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(out_channel, out_channel//2)
        self.linear_2 = nn.Linear(out_channel//2, num_classes)


    def forward(self, x):
        output = self.embedding(x)

        output = output.permute(0, 2, 1)

        output = self.conv_1(output)
        output = self.relu(output)
        output = self.pool(output)

        output = self.conv_2(output)
        output = self.relu(output)
        output = self.pool(output)

        output = torch.max(output, axis=2).values

        output = self.linear_1(output)
        output = self.relu(output)

        output = self.linear_2(output)
        output = F.sigmoid(output)
        return output

Параметры обучения сети

In [ ]:
num_classes = 1

epochs = 10
print_batch_n = 100

In [ ]:
model = Net(vocab_size = max_words)

In [ ]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

Net(
  (embedding): Embedding(10000, 128)
  (conv_1): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (linear_1): Linear(in_features=128, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=1, bias=True)
)
Parameters: 1370497


Определяем оптимизатор и функцию потерь для обучения сети

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=10e-3)
criterion = nn.BCELoss()

Обучаем сеть

In [ ]:
model.train()
threshold = 0.5 #пороговое значение (на выходе слоя сигмоиды) для определения токсичности высказывания (токсисные кодируются 1)

train_loss_history = []
test_loss_history = []

for epoch in range(epochs):
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        loss = loss.item()
        running_items += len(labels)

        pred_labels = torch.squeeze((outputs > threshold).int())
        running_right += (labels == pred_labels).sum()


    print(f'Epoch {epoch + 1} / {epochs}. ' \
       f'Loss: {loss: .3f}. ' \
       f'Acc: {running_right / running_items: .3f}. ')

    train_loss_history.append(loss)

Epoch 1 / 10. Loss:  0.662. Acc:  0.611. 
Epoch 2 / 10. Loss:  0.554. Acc:  0.612. 
Epoch 3 / 10. Loss:  0.420. Acc:  0.690. 
Epoch 4 / 10. Loss:  0.266. Acc:  0.893. 
Epoch 5 / 10. Loss:  0.116. Acc:  0.948. 
Epoch 6 / 10. Loss:  0.081. Acc:  0.974. 
Epoch 7 / 10. Loss:  0.042. Acc:  0.988. 
Epoch 8 / 10. Loss:  0.013. Acc:  0.994. 
Epoch 9 / 10. Loss:  0.009. Acc:  0.997. 
Epoch 10 / 10. Loss:  0.006. Acc:  0.998. 


Точность модели на тестовой выборке

In [ ]:
test_dataset = DataWrapper(x_test, df_test['label'].values)
test_loader = DataLoader(test_dataset, batch_size=200)

In [ ]:
model.eval()

test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
for j, data in enumerate (test_loader):
    test_outputs = model(data[0])
    test_labels = data[1]

    test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
    test_running_total += len(data[1])
    pred_test_labels = torch.squeeze((test_outputs > threshold).int())
    test_running_right += (test_labels == pred_test_labels).sum()

print(f'Test loss: {test_loss: .3f}. Test acc: {test_running_right / test_running_total: .3f}')
test_loss_history.append(test_loss.item())

Test loss:  1.227. Test acc:  0.836
